# Лабораторная работа 9
# Создание текстового файла большого объема с помощью генераторной функции

#### Кураленко Алексей 01.06

In [1]:
from random import randint, choice

In [2]:
?randint

Signature: randint(a, b)
Docstring:
Return random integer in range [a, b], including both end points.
        
File:      c:\users\lenovo\anaconda3\envs\widgets_env\lib\random.py
Type:      method

In [4]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
f'Код символа: {(c:=randint(*latin_code))}, символ: {chr(c)}'

'Код символа: 117, символ: u'

# Функция process_time из модуля time

In [5]:
from time import process_time

In [6]:
?process_time


Docstring:
process_time() -> float

Process time for profiling: sum of the kernel and user-space CPU time.
Type:      builtin_function_or_method

In [17]:
start = process_time()
[x**2 for x in range(1_000_000)]
end = process_time()
print(f'{end - start} секунд')

0.1875 секунд


# Задание 9.1. Запись данных в текстовый файл

In [18]:
file_name = 'test.txt'
f = open(file_name, 'w')

In [19]:
lines_list = [f'{x}\n' for x in 'test']
lines_list

['t\n', 'e\n', 's\n', 't\n']

In [20]:
f.writelines(lines_list)

In [21]:
f.close()

In [22]:
with open(file_name,'r') as f:
    for line in f:
        print(line, end='')

t
e
s
t


In [23]:
lines_gen_expr = (f'{x}\n' for x in range(10**6))
def lines_gen_fun(number_lines=10**6):
    yield from (f'{x}\n' for x in range(number_lines))
with open(file_name,'w') as f:
    f.writelines(lines_gen_expr)
    f.writelines(lines_gen_fun())

In [25]:
def lines_gen_fun(number_lines = 10**6):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
    print(f'{file_size/1024**2} Mb')

with open(file_name,'w') as f:
    f.writelines(lines_gen_fun())


6.569757461547852 Mb


In [26]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun()) 
    end = process_time()
    print(end - start, 'секунд')

6.569757461547852 Mb
1.21875 секунд


In [30]:
def lines_gen_fun(number_lines=10**6, status=False):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
        # отображение статуса записи в файл в процентах
        if status:
            status_number = x / number_lines * 100
            # \r — возврат курсора в начало строки
            print(f'\r{int(status_number)}%', end='', flush=True)
    print(f'\n {file_size/1024**2} Mb')

In [31]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun(3*10**3, status=True)) 
    end = process_time()
    print(end - start, 'секунд')

99%
 0.013246536254882812 Mb
1.546875 секунд


# Задание 9.2. Генерация строк

In [32]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
number_words = (10, 20)
number_symbols = (5, 10)

In [33]:
len_word = randint(*number_symbols)
word = ''.join([chr(randint(*latin_code)) for i in range(len_word)])
word

'ifqrxkr'

In [40]:
number_code=(ord('0'), ord('9'))
kir_code=(ord('А'), ord('Я'))

skroka=' '.join(''.join([chr(randint(*choice([number_code,kir_code,latin_code]))) for i in range(randint(*number_symbols))]) for j in range(*number_words))

In [41]:
skroka

'2ХxРУ82e mБec60rz tГmЕi3 phМЕЭs ЪЕyГЗ3n9 УnЬН3bЬ 03bМВ41ПЩТ З4ИГБqЮ7Н Д5ry0 htpnl'

In [43]:
def lines_gen_fun_v2(file_size_mb, type_symbols, number_words, number_symbols):
    size = 0
    limit_bytes = file_size_mb * 1024 ** 2   
    while True:
        line = ' '.join(''.join(chr(randint(*type_symbols)) 
                for _ in range(randint(*number_symbols)))
            for _ in range(randint(*number_words)))
        size += len(line) + 1 
        if size > limit_bytes:
            return
        yield line + '\n'

In [44]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun_v2(1.54,latin_code, number_words, number_symbols )) 
    end = process_time()
    print(end - start, 'секунд')

2.734375 секунд


# Задание 9.3. Запись сгенерированных строк в текстовый файл

In [57]:
from random import randint
from time import process_time

def file_gen(file_name: str,
             file_size: int,
             type_symbols: str = 'latin',
             number_words: (int, int) = (10, 20),
             number_symbols: (int, int) = (5, 10)):
    """
    Генерирует файл с текстовыми строками случайных слов из заданного набора символов.
    Каждая строка содержит случайное количество слов, каждое слово — случайное количество символов.
    Запись продолжается до достижения заданного размера файла в мегабайтах.

    Args:
        file_name (str): имя файла для записи.
        file_size (int): желаемый размер файла в мегабайтах.
        type_symbols (str): набор символов для генерации ('latin', 'digits', 'cyrillic').
        number_words (tuple[int, int]): диапазон количества слов в строке (min, max).
        number_symbols (tuple[int, int]): диапазон количества символов в слове (min, max).
    """
    size = 0  # текущий размер записанных данных в байтах
    limit_bytes = file_size * 1024 ** 2  # лимит размера файла в байтах (конвертация из мегабайт)
    start = process_time()  # время начала генерации и записи

    symbol_sets = {
        'latin': [(ord('A'), ord('Z'))],     # диапазон кодов для латинских заглавных букв
        'digits': [(ord('0'), ord('9'))],    # диапазон кодов для цифр
        'cyrillic': [(ord('А'), ord('Я'))]   # диапазон кодов для кириллицы
    }

    last_percent = -1  # последняя выведенная процентная отметка

    with open(file_name, 'w', encoding='utf-8') as f:  # открываем файл для записи в кодировке UTF-8
        # перебираем строки, которые генерирует внешний генератор lines_gen_fun_v2
        for line in lines_gen_fun_v2(file_size, *symbol_sets[type_symbols], number_words, number_symbols):
            size += len(line)  # увеличиваем размер на длину текущей строки в байтах
            f.write(line)      # записываем строку в файл

            status_number = int(size / limit_bytes * 100)  # вычисляем процент выполнения
            if status_number > last_percent:  # если прогресс изменился, выводим обновление
                print(f'\r{status_number}%', end='', flush=True)  # печатаем прогресс в консоль на той же строке
                last_percent = status_number  # обновляем последнюю сохранённую отметку прогресса

    end = process_time()  # время окончания записи

    print("\n")  # перевод строки после прогресса
    print(end - start, 'секунд')  # выводим, сколько заняло времени (в секундах)
    print(f'\n{size / 1024 ** 2} Mb')  # выводим фактический размер записанного файла в мегабайтах

In [58]:
file_gen(file_name, 2, 'latin')

99%

1.71875 секунд

1.9999637603759766 Mb
